In [1]:
from bilibili_api import user,Credential,settings
import json

settings.proxy = "http://127.0.0.1:7890"
with open('my_redential.json','r') as f:
    cre_param=json.load(f)
CRE=Credential(**cre_param)

In [2]:
## user resp example
UID=176421741
u=user.User(uid=176421741,credential=CRE)
resp=await u.get_followers()
resp


ResponseCodeException: 接口返回错误代码：-352，信息：-352。
{'code': -352, 'message': '-352', 'ttl': 1}

In [ ]:

u=user.User(uid=UID,credential=CRE)
resp=await u.get_followings()
resp=await u.get_all_followings()
resp=await u.get_followers()
resp=await u.get_relation_info()
# resp=await u.get_user_info()   #! -352 error *
resp

In [ ]:
from pyvis.network import Network
from typing import Literal
from bilibili_api.errors import ResponseCodeException
import networkx as nx
from tqdm import tqdm
import random

class UserGraph(nx.DiGraph):

    def __init__(self,credential=CRE,incoming_graph_data=None, **attr):
        super().__init__(incoming_graph_data, **attr)
        self.cre=credential

    @staticmethod
    def _attr_lst_mapping(attr_lst:list[dict],key_map:dict[str,str]):
        mapped_attr_lst=[]
        for old_key,new_key in key_map.items():
            for attr_dct in attr_lst:
                attr_copy=attr_dct.copy()
                if old_key in attr_copy:
                    attr_copy[new_key]=str(attr_copy[old_key])
                    mapped_attr_lst.append(attr_copy)
        return mapped_attr_lst
    
    @staticmethod
    def _attr_filter(attr_dct:dict,keys=['mid','uname','folloer','following']):
        return {k:v for k,v in attr_dct.items() if k in keys}

    def _as_pyvis(self) -> Network:
        pv_net=Network(directed=True,notebook=True,cdn_resources='in_line')
        pv_net.from_nx(self)
        return pv_net
    def save_html(self,path='graph.tmp.html'):
        pv_net=self._as_pyvis()
        pv_net.toggle_physics(True)
        pv_net.show_buttons(filter_=['physics'])
        return pv_net.show(path)
    
    def add_relation_list(self,
                      source,
                      follow_type:Literal['->','<-'],
                      uids=[]
                      ):
        if follow_type=='->':
            edges_to_add = [(source, uid) for uid in uids]
        elif follow_type=='<-':
            edges_to_add = [(uid, source) for uid in uids]
        else:
            raise ValueError
        
        self.add_edges_from(edges_to_add)

    def add_user(self,info_with_uid:list[dict]):
        for info_dct in info_with_uid:
            self.add_node(info_dct['mid'],**info_dct)
    
    def add_relation(self,u_info:int|dict,v_info:int|dict):
        u=u_info if isinstance(u_info,int) else u_info['mid']
        v=v_info if isinstance(v_info,int) else v_info['mid']
        self.add_edge(u,v)
        if isinstance(u_info,dict):
            self.nodes[u].update(u_info)
        if isinstance(v_info,dict):
            self.nodes[v].update(v_info)
    
    def info_as_title(self,sep:str='\n',keys=('mid','uname','following','follower')):
        for node,attr in self.nodes(data=True):
            l=[f'{k}={attr[k]}' for k in keys if k in attr]
            title=sep.join(l)
            self.nodes[node]['title']=title
            # self.add_node(node,title=title)
    def info_as_label(self,sep:str=',',keys=['uname']):
        for node,attr in self.nodes(data=True):
            l=[f'{attr[k]}' for k in keys if k in attr]
            label=sep.join(l)
            self.nodes[node]['label']=label

    async def update_follow_info(self):
        for node in self.nodes:
            u=user.User(node,credential=self.cre)
            resp=await u.get_relation_info()
            info={k:v for k,v in resp.items() if k in ('uname','following','follower')}
            self.add_node(node,**info)

    async def extend_relations(self,max_user=100):
        fler_list=[]
        flin_list=[]
        k = min(max_user,len(self.nodes))
        nodes=random.sample(list(self.nodes),k)
        for node in tqdm(nodes,desc='requesting',unit='user',miniters=0):
            u=user.User(node,credential=self.cre)
            try:
                fler_resp=await u.get_followers()
                flin_resp=await u.get_followings()
            except ResponseCodeException as e:
                # print(f'{node}:{e.msg}')
                continue
            for fler_dct in fler_resp['list']:
                fler_list.append(self._attr_filter(fler_dct))
            for flin_dct in flin_resp['list']:
                flin_list.append(self._attr_filter(flin_dct))
        for fler_dct in tqdm(fler_list,desc='adding followers',unit='user'):
            self.add_relation(fler_dct,node)
        for flin_dct in tqdm(flin_list,desc='adding followings',unit='user'):
            self.add_relation(node,flin_dct)


available attr in pyvis: lable title group(int) size(int)

In [ ]:
## add edges
g=UserGraph()

g.add_node(176421741)
await g.extend_relations()
await g.extend_relations()
await g.extend_relations(max_user=300)
print('-'*20)
await g.update_follow_info()
print('*'*20)


In [ ]:
## viz
g.info_as_label()
g.info_as_title()
g.save_html()

In [ ]:
nx.write_gexf(g,'graph.tmp.gexf')